## Environment creation for tutor experiment

When creating the files used in the experiment this code had a bug resulting in incorrect values for expected reward ("expReward"). Since the expReward values are unused in the experiment the error had no negative impact.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import json
from tqdm import tqdm

from src.utils.experiment_creation import create_structure, create_tree, create_init, generate_samples, create_structure_po

In [ ]:
def create_env_structure(COLUMNS=4, BLOCKS=2, dir="./data/environments"):
    SIZE = COLUMNS*BLOCKS*7 + COLUMNS
    NAME = f"{COLUMNS}_{SIZE}"

    JSON_PATH = f"{dir}/experiment_instances/{NAME}_json"
    STRUCTURE_PATH = f"{dir}/structure_files"

    try:
        os.mkdir(JSON_PATH)
    except:
        print("Directory already exists")
    
    TREE = create_tree(columns=COLUMNS, blocks=BLOCKS)
    INIT = create_init(columns=COLUMNS, blocks=BLOCKS)

    env = create_structure(columns=COLUMNS, blocks=BLOCKS)
    with open(f"{STRUCTURE_PATH}/{NAME}_structure.json", "w") as f:
        json.dump(env, f)
    
    env_po = create_structure_po(TREE, INIT)
    with open(f"{STRUCTURE_PATH}/{NAME}_po_structure.json", "w") as f:
        json.dump(env_po, f)

    param_mapping = json.load(open("./data/environments/params.json"))
    param_mapping = {int(k):tuple(v) for k,v in param_mapping.items()}
    COSTS = [param_mapping[i][0] for i in range(len(param_mapping))]
    TAUS = [param_mapping[i][1] for i in range(len(param_mapping))]

    SEED = 0
    MAX_ACTIONS = 200

    for j, (COST, TAU) in tqdm(enumerate(zip(COSTS, TAUS))):
        env_data, index = generate_samples(j, COST, TAU, TREE, INIT, SEED, MAX_ACTIONS)
        filename = f"/{index}_{str(COST)[2:]}_{str(TAU)[2:]}"
        # Uncomment to save env data
        # with open(PATH+filename+".p", mode="wb") as f:
        #     pickle.dump(env_data, f)

        # Create json
        trial_data = []
        for i in range(len(env_data["envs"])):
            env = env_data["envs"][i]
            trial = {
                "trial_id": i,
                "stateRewards": env["ground_truth"].tolist(),
                "samples": env["samples"],
                "expReward": env["mp_exp_reward"],
                "n_actions": env["mp_actions"]
            }
            trial_data.append(trial)
        
        with open(JSON_PATH+filename+".json", 'w') as outfile:
            json.dump(trial_data, outfile)

In [ ]:
create_env_structure(1, 1)

In [ ]:
create_env_structure(2, 1)

In [ ]:
create_env_structure(2, 2)

In [ ]:
create_env_structure(4, 2)